<a href="https://colab.research.google.com/github/valeriu-balaban/improving-robustness-with-variance-penalization/blob/main/improving_robustness_with_variance_penalization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

**Please run this notebook on a runtime with a GPU. Select Runtime >> Change runtime type >> Hardawre accelerator: GPU**

Check if a GPU is available using by running the next cell.

In [2]:
!nvidia-smi

Tue Oct 11 21:04:01 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
!git clone https://github.com/valeriu-balaban/improving-robustness-with-variance-penalization.git

Cloning into 'improving-robustness-with-variance-penalization'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 25 (delta 7), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), done.


In [3]:
%%capture
!pip install wandb pytorch-lightning

Runtime info

In [4]:
import torch
import wandb
import multiprocessing
import pytorch_lightning as pl


print("Pytorch", torch.__version__)
print("WandB", wandb.__version__)
print("Pytorch Lightning", pl.__version__)
print("–"*40)
print("Number of CPUs", multiprocessing.cpu_count())
print("–"*40)
!nvidia-smi

Pytorch 1.12.1+cu113
WandB 0.13.4
Pytorch Lightning 1.7.7
––––––––––––––––––––––––––––––––––––––––
Number of CPUs 2
––––––––––––––––––––––––––––––––––––––––
Tue Oct 11 21:30:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+----------

In [5]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Training with Limited Data using Variance Minimization

To train with limited data set the following parameters:
- `noise_rate = 0.0`
- `asym_noise = 0`
- `imbalanced = True`
- `train_ratio` adjust based on the dataset used
- `lambda_2` sets the value of $\lambda$ for `FindQ` procedure when using "distributional-varpen" as a loss function
- `lambda_3` sets the value of $\gamma$ for `FindQ` procedure when using "distributional-varpen" as a loss function

In [10]:
import wandb

sweep_config = {
  "name": "limited data sweep",
  "program": "improving-robustness-with-variance-penalization/train.py",
  "method": "grid",
  "parameters": {
      "loss_function":    { "values": ["distributional-varpen", "direct-varpen",]},
      "noise_rate":       { "values": [0.0, ]},
      # "random_seed":      { "values": [1, 2, 3, 4, 5]},
      # "random_seed":      { "values": [1, 2, 3]},
      "random_seed":      { "values": [1, ]},
      "asym_noise":       { "values": [0, ]},

      "dataset":          { "values": ["CIFAR-10"]},
      "train_ratio":      { "values": [0.01, 0.02, 0.03, 0.04]},

      # "dataset":          { "values": ["MNIST"]},
      # "train_ratio":      { "values": [ 0.002, 0.004, 0.006, 0.008]},

      "lambda_2":         { "values": [0.75 ]},
      # "lambda_2":         { "values": [0.10000000149011612, 0.1333521455526352, 0.17782793939113617, 0.23713737726211548, 0.3162277638912201, 0.4216965138912201, 0.5623413324356079, 0.7498942017555237, 1.0, 1.3335214853286743, 1.778279423713684, 2.3713736534118652, 3.1622776985168457]},
      
      # "lambda_3":         { "values": [0, ]},
      "lambda_3":         { "values": [0.45,]},
      "lambda_4":         { "values": [0, ]},

      "normalize":        { "values": [False,]},
      "imbalanced":       { "values": [True,]},
  }
}

wandb.sweep(sweep_config, project="weighted-mean-robustness")

Create sweep with ID: mh9no285
Sweep URL: https://wandb.ai/vbalaban/weighted-mean-robustness/sweeps/mh9no285


'mh9no285'

In [ ]:
!wandb agent vbalaban/weighted-mean-robustness/mh9no285 

# Training with Mislabeled Data using Variance Maximization

To train with mislabeled data set the following parameters:
- `noise_rate` sets the percentage of mislabeld samples
- `asym_noise = 0`
- `imbalanced = False`
- `train_ratio = 0.9`
- `lambda_2` sets the value of $\lambda$ for `FindQ` procedure when using "distributional-varpen" as a loss function
- `lambda_3` sets the value of $\gamma$ for `FindQ` procedure when using "distributional-varpen" as a loss function

In [2]:
import wandb

sweep_config = {
  "name": "Mislabeled data sweep",
  "program": "improving-robustness-with-variance-penalization/train.py",
  "method": "grid",
  "parameters": {
      "loss_function":    { "values": [ "distributional-varpen",]},
      "noise_rate":       { "values": [0.2, 0.4, 0.6]},
      # "random_seed":      { "values": [1, 2, 3, 4, 5]},
      # "random_seed":      { "values": [1, 2, 3]},
      "random_seed":      { "values": [1, ]},
      "asym_noise":       { "values": [0,]},

      "dataset":          { "values": ["MNIST"]},
      "train_ratio":      { "values": [0.9, ]},

      # "lambda_2":         { "values": [-0.5, -0.7, -0.9,]},
      "lambda_2":         { "values": [-0.10000000149011612, -0.11787686496973038, -0.13894955813884735, -0.16378936171531677, -0.19306977093219757, -0.2275845855474472, -0.2682695984840393, -0.3162277638912201, -0.3727593719959259, -0.43939706683158875, -0.5179474353790283, -0.6105402112007141, -0.7196856737136841, -0.8483428955078125, -1.0]},
      # "lambda_2":         { "values": [0.10000000149011612, 0.1333521455526352, 0.17782793939113617, 0.23713737726211548, 0.3162277638912201, 0.4216965138912201, 0.5623413324356079, 0.7498942017555237, 1.0, 1.3335214853286743, 1.778279423713684, 2.3713736534118652, 3.1622776985168457]},
      
      "lambda_3":         { "values": [1, ]},
      "lambda_4":         { "values": [0, ]},

      "normalize":        { "values": [False,]},
      "imbalanced":       { "values": [False,]},
  }
}

wandb.sweep(sweep_config, project="weighted-mean-robustness")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: bzbr06h6
Sweep URL: https://wandb.ai/vbalaban/weighted-mean-robustness/sweeps/bzbr06h6


'bzbr06h6'

In [ ]:
!wandb agent vbalaban/weighted-mean-robustness/bzbr06h6 